<a href="https://colab.research.google.com/github/themendu/IPL-Predictor/blob/master/tabnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytorch-tabnet


In [2]:
import pandas as pd
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
final=pd.read_csv('/content/drive/My Drive/changed_data.csv')

In [5]:
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from pytorch_tabnet.tab_model import TabNetClassifier

from sklearn.model_selection import KFold

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [6]:
final.drop(columns=['Unnamed: 0'],inplace=True)

In [24]:
final.shape

(519, 72)

In [14]:
from sklearn.model_selection import train_test_split
def datasplitter(data):
    train_data, val_data = train_test_split(data, test_size = 0.17, random_state = 9)
    y_train = train_data['winner'].values
    y_val = val_data['winner'].values
    x_train = train_data.drop(['winner'], axis = 1).values
    x_val = val_data.drop(['winner'], axis = 1).values
    return y_train,y_val,x_train,x_val


y_train,y_val,x_train,x_val=datasplitter(final)

In [15]:
x_train.shape

(430, 71)

In [16]:
y_train.shape

(430,)

In [17]:
from pytorch_tabnet.tab_model import TabNetClassifier

model = TabNetClassifier(n_d=16,#larger,then overfit
                       n_a=16,
                       n_steps=4,
                       gamma=1.3,
                       n_independent=2,
                       n_shared=2,
                       device_name='cpu',
                       optimizer_params=dict(lr=1e-3),
                       optimizer_fn=torch.optim.Adam)

Device used : cpu


In [21]:
model.fit(X_train = x_train,
             y_train = y_train,
             X_valid = x_val,
             y_valid = y_val,
             max_epochs = 100,
             batch_size=32,
             drop_last=False,
             patience =50)

Will train until validation stopping metric hasn't improved in 50 rounds.
---------------------------------------
| EPOCH |  train  |   valid  | total time (s)
| 1     | 0.50102 |  0.46512 |   0.5       
| 2     | 0.53344 |  0.52401 |   1.0       
| 3     | 0.50866 |  0.44742 |   1.6       
| 4     | 0.54633 |  0.52022 |   2.1       
| 5     | 0.53232 |  0.52022 |   2.6       
| 6     | 0.51271 |  0.52528 |   3.1       
| 7     | 0.51654 |  0.51946 |   3.6       
| 8     | 0.49002 |  0.52351 |   4.2       
| 9     | 0.52961 |  0.51744 |   4.7       
| 10    | 0.54215 |  0.50961 |   5.2       
| 11    | 0.48122 |  0.50531 |   5.7       
| 12    | 0.52307 |  0.49823 |   6.2       
| 13    | 0.50494 |  0.48913 |   6.7       
| 14    | 0.58958 |  0.48155 |   7.2       
| 15    | 0.56647 |  0.41709 |   7.8       
| 16    | 0.53995 |  0.50784 |   8.3       
| 17    | 0.57263 |  0.46562 |   8.8       
| 18    | 0.49182 |  0.52098 |   9.3       
| 19    | 0.51623 |  0.48458 |   9.8       
| 20

In [22]:
from sklearn.metrics import accuracy_score

y_oof = model.predict(x_train)

print(accuracy_score(y_train,y_oof))

0.6046511627906976


In [23]:
y_oof = model.predict(x_val)

print(accuracy_score(y_val,y_oof))

0.550561797752809
